In [2]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.infogram import H2OInfogram
from h2o.estimators.gbm import H2OGradientBoostingEstimator
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import EvaluationModule
pd.options.mode.chained_assignment = None

In [5]:
dataset = CoronnaCERTAINDataset(
    library_root = '../Dataset/',
    challenge = 'binary_classification', #option: regression, regression_delta, classification, binary_classification, regression_delta_binary
    dataset = 'CORRONA CERTAIN', 
    process_approach = 'SC', #option: KVB, SC
    imputation = 'IterativeImputer', #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group = ['bionaive TNF'], #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group = 'all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points = (0,3), 
    train_test_rate = 0.6,
    remove_low_DAS = False,
    save_csv = False, 
    random_state = 2022,
    verbose=False)

In [126]:
import pandas as pd 
df=pd.read_csv(r'C:\Users\buz galbraith\Desktop\reaserch\ML_RA_EHR\Dataset\Coronna_Data_CERTAIN_raw.csv')

In [127]:
df_dev_demo = df.loc[:, :"hx_anycancer"]
grp = df_dev_demo.set_index('futime').groupby('UNMC_id')
groups = [g.reindex(range(0, 15, 3)).reset_index() for _, g in grp]
demo_out = pd.concat(groups, ignore_index=True).reindex(
df_dev_demo.columns, axis=1)

        # time_varying features
df_dev_time_varying = df.loc[:, "seatedbp1":]
df_dev_time_varying['UNMC_id'] = df['UNMC_id']
df_dev_time_varying['futime'] = df['futime']
grp = df_dev_time_varying.set_index('futime').groupby('UNMC_id')
groups = [g.reindex(range(0, 15, 3)).reset_index() for _, g in grp]
time_varying_out = pd.concat(groups, ignore_index=True).reindex(
df_dev_time_varying.columns, axis=1)
time_varying_out['UNMC_id'] = time_varying_out['UNMC_id']
time_varying_out = time_varying_out.drop(columns=['futime', 'UNMC_id'])
out_final = pd.concat([demo_out, time_varying_out], axis=1)

In [128]:
out_final.columns
out_final=out_final[out_final['futime']==3]
out_final=out_final[out_final['init_group']=='bionaive TNF']
out_final=out_final[['tender_jts_28','swollen_jts_28','usresultsCRP','md_global_assess','pt_global_assess']]

# #
#out_final=out_final.drop(['statin_use','Unnamed: 62','Unnamed: 63'],axis=1)

out_final


,tender_jts_28,swollen_jts_28,usresultsCRP,md_global_assess,pt_global_assess
11,0.0,2.0,0.31,10.0,20.0
76,0.0,0.0,0.84,1.0,5.0
81,1.0,3.0,0.69,20.0,20.0
86,5.0,8.0,0.33,25.0,20.0
106,14.0,1.0,2.12,40.0,50.0
...,...,...,...,...,...
6106,2.0,2.0,21.52,25.0,85.0
6111,1.0,2.0,0.71,20.0,0.0
6131,0.0,0.0,0.18,5.0,2.0
6136,2.0,4.0,3.33,50.0,20.0


In [129]:
d=dict()
for col in out_final.columns:
    print(col)
    if(len(out_final[col].value_counts())>10):
        d[col]={"non-null":410    -(410     -sum(out_final[col].value_counts())),'null':410     -sum(out_final[col].value_counts())}
    else:
        d[col]=out_final[col].value_counts()  
        d[col]["null"]=410    -sum(out_final[col].value_counts())

tender_jts_28
swollen_jts_28
usresultsCRP
md_global_assess
pt_global_assess


In [123]:
with open('test6.csv', 'w') as f:
    for e in d:
        f.write("%s,\n" % (e))
        for i in d[e].keys():
            temp=(d[e][i]/410 )*100
            f.write("\t,%s, %s(%s),\n" % (i, d[e][i],temp))
        f.write('\n')

In [125]:
(1/435)*100

0.22988505747126436